In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_json,col
from pyspark.sql.types import *
from os.path import abspath

spark = SparkSession\
        .builder\
        .appName("pyspark-notebook")\
        .master("spark://spark-master:7077")\
        .config("spark.executor.memory", "512m")\
        .config("hive.metastore.uris", "thrift://hive-metastore:9083")\
        .config("spark.sql.warehouse.dir", "/user/hive/warehouse")\
        .enableHiveSupport()\
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

23/10/12 20:52:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## 3. The Data

In [8]:
!apt install wget -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpcre2-8-0
The following NEW packages will be installed:
  libpcre2-8-0 wget
0 upgraded, 2 newly installed, 0 to remove and 0 not upgraded.
Need to get 1115 kB of archives.
After this operation, 3925 kB of additional disk space will be used.
Get:1 http://deb.debian.org/debian buster/main amd64 libpcre2-8-0 amd64 10.32-5 [213 kB]
Get:2 http://deb.debian.org/debian buster/main amd64 wget amd64 1.20.1-1.1 [902 kB]
Fetched 1115 kB in 1s (2034 kB/s)
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package libpcre2-8-0:amd64.
(Reading database ... 27731 files and directories currently installed.)
Preparing to unpack .../libpcre2-8-0_10.32-5_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 11%] [######....................

In [11]:
!wget 'https://drive.google.com/uc?export=download&id=1wdytBkoYRyaNNv3XUNnwcGbdNUSfok7u' -O uplift_data.zip

--2023-10-12 21:00:02--  https://drive.google.com/uc?export=download&id=1wdytBkoYRyaNNv3XUNnwcGbdNUSfok7u
Resolving drive.google.com (drive.google.com)... 64.233.165.194, 2a00:1450:4010:c07::c2
Connecting to drive.google.com (drive.google.com)|64.233.165.194|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hq6bruqkegk5tburup1p8u8phqksmplp/1697144400000/14904333240138417226/*/1wdytBkoYRyaNNv3XUNnwcGbdNUSfok7u?e=download&uuid=8924b671-6f93-4378-9805-1f405a280e95 [following]
--2023-10-12 21:00:03--  https://doc-0c-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hq6bruqkegk5tburup1p8u8phqksmplp/1697144400000/14904333240138417226/*/1wdytBkoYRyaNNv3XUNnwcGbdNUSfok7u?e=download&uuid=8924b671-6f93-4378-9805-1f405a280e95
Resolving doc-0c-c0-docs.googleusercontent.com (doc-0c-c0-docs.googleusercontent.com)... 216.58.211.225, 216.58.209.193, 2a

In [13]:
!unzip uplift_data.zip

Archive:  uplift_data.zip
  inflating: clients.csv             
  inflating: uplift_test.csv         
  inflating: uplift_train.csv        


In [14]:
ls

 clients.csv                                                  uplift_data.zip
 pyspark.ipynb                                                uplift_test.csv
'uc?export=download&id=1wdytBkoYRyaNNv3XUNnwcGbdNUSfok7u'     uplift_train.csv
'uc?export=download&id=1wdytBkoYRyaNNv3XUNnwcGbdNUSfok7u.1'


In [21]:
data = spark.read.csv(path="clients.csv", sep=",", header=True)

In [22]:
data.count()

400162

In [23]:
data.head()

Row(client_id='000012768d', first_issue_date='2017-08-05 15:40:48', first_redeem_date='2018-01-04 19:30:07', age='45', gender='U')

In [25]:
df1 = data.select(["client_id", "age", "gender"])

In [27]:
df1.head()

Row(client_id='000012768d', age='45', gender='U')

In [28]:
df1 = data.select(["client_id", "age"])

In [29]:
cols_description = df1.filter(df1['age'] == '45')

In [30]:
df1 = df1.join(other=cols_description, on=['age'], how='left_anti')

In [35]:
df1 = df1.withColumnRenamed("client_id", "c1").withColumnRenamed("age", "c2").withColumnRenamed("gender", "c3")

In [36]:
df1.write.saveAsTable('hive_table')

In [37]:
ndf= spark.sql("""
select * from hive_table
""")


In [38]:
ndf.show(10)

+---+----------+
| c2|        c1|
+---+----------+
| 72|000036f903|
| 68|000048b7a6|
| 60|000073194a|
| 67|00007c7133|
| 46|0000b59cec|
| 36|0000bb4e4e|
| 34|0000bcec9c|
| 70|0000eecb82|
| 56|0000f0ecdb|
| 83|00010925a5|
+---+----------+
only showing top 10 rows

